In [ ]:
# default_exp core

# Fire Split

> Split individual fire events from tif files

In [ ]:
# hide
from nbdev.export import notebook2script

In [ ]:
# export
import rasterio
import numpy as np
from tqdm import tqdm
import scipy.ndimage as ndimage
import matplotlib.pyplot as plt
from pathlib import Path
import pandas as pd

In [ ]:
# export
def read_data(filename):
    data = rasterio.open(filename)
    meta = data.meta
    data = data.read().squeeze()
    data[data<0] = 0
    return data, meta

def save_data(filename, raster, meta):
    meta['nodata'] = 0
    meta['dtype'] = 'uint32'
    with rasterio.open(filename, 'w', **meta) as dst:
        dst.write(raster[None].astype('uint32'))

def split_fires(data, interval_days=8):
    data = data.astype(np.uint32)

    labels, _ = ndimage.label(data > 0, structure=np.ones((3,3)))

    unique_pairs = np.unique(np.array((data, labels), dtype=np.uint32).reshape(2,-1), axis=1)

    df = pd.DataFrame({'date': unique_pairs[0], 'label': unique_pairs[1]})
    df_overlap = df.groupby('label').agg(lambda x : len(x))
    overlap_index = df_overlap.loc[df_overlap.date>1].index
    df = df.groupby('label').agg(lambda x : np.array(x))
    df = df.loc[overlap_index]

    df['splits'] = [np.diff(df.iloc[i].date) > interval_days for i in range(len(df))]

    for k in range(len(df)):
        df_row = df.iloc[k]
        for i, s in enumerate(df_row.splits):
            if s:
                split1 = df_row.date[i]
                split2 = df_row.date[i+1]
                I1 = (labels==df_row.name) & (data==split1)
                I2 = (labels==df_row.name) & (data==split2)
                l1, _ = ndimage.label(I1, structure=np.ones((3,3)))
                l2, _ = ndimage.label(I2, structure=np.ones((3,3)))
                l1[l1>0] += labels.max()
                labels[I1] = l1[I1]
                l2[l2>0] += labels.max()
                labels[I2] = l2[I2]

    labels2 = np.zeros_like(labels)
    rand_labels = np.random.permutation(list(range(len(np.unique(labels)[1:]))))
    for i, l in enumerate(np.unique(labels)[1:]):
        labels2[labels==l] = rand_labels[i]
    return labels2

def run_all(input_path, save_path, interval_days=8):
    if input_path.is_dir():
        files = list(path.iterdir())
        files = [f for f in files if 'tif' in f.suffix]
    else:
        files = [input_path]
    
    for f in tqdm(files):
        fsave = save_path/f'{f.stem}_labels.tif'
        data, meta = read_data(f)
        labels = split_fires(data, interval_days=interval_days)
        save_data(fsave, labels, meta)

In [ ]:
# hide
notebook2script()

Converted 00_core.ipynb.
Converted 01_cli.ipynb.
Converted index.ipynb.
